### Урок 2. #Профилирование пользователей. Сегментация аудитории: unsupervised learning (clustering, LDA/ARTM), supervised (multi/binary classification)#

План занятия:

1. задача сегментации аудитории по интересам - для чего
2. тематическое моделирование - получаем эмбединги текстов
3. решаем downstream-задачу (профилирование аудитории новостного портала)

### Переходим к практике

In [1]:
import pandas as pd

Наши новости

In [2]:
news = pd.read_csv("articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


Загрузим пользователей и списки последних прочитанных новостей

In [3]:
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


Итак, нам нужно получить векторные представления пользователей на основе прочитанным ими новостей и самих новостей

### 1. Получаем векторные представления новостей

In [4]:
#from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
from gensim import models

In [5]:
#предобработка текстов
import re
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
#from nltk.tokenize import word_tokenize

from razdel import tokenize # https://github.com/natasha/razdel
#!pip install razdel

import pymorphy2  # pip install pymorphy2

[nltk_data] Downloading package stopwords to /home/helgi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
#sw = nltk.download('stopwords')
stopword_ru = stopwords.words('russian')
#stopword_ru = sw.words('russian')
len(stopword_ru)

morph = pymorphy2.MorphAnalyzer()

In [7]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)


776

In [8]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [11]:
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

CPU times: user 20.7 s, sys: 907 ms, total: 21.6 s
Wall time: 21.7 s


In [12]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

CPU times: user 2min 34s, sys: 150 ms, total: 2min 34s
Wall time: 2min 34s


А теперь в 3 строчки обучим нашу модель

In [13]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

Что такое common_dictionary и как он выглядит

In [14]:
common_dictionary[10]

'ватутин'

Все просто - это словарь наших слов

Запускаем обучение

In [15]:
%%time
from gensim.models import LdaModel
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)#, passes=10)

CPU times: user 1min 21s, sys: 3min 35s, total: 4min 56s
Wall time: 42.4 s


In [16]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model1.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

In [17]:
lda[common_corpus[3]]

[(1, 0.1457878),
 (8, 0.18848033),
 (11, 0.17453343),
 (19, 0.3147755),
 (23, 0.16633725)]

Предыдущее значение: [(1, 0.20295064), (6, 0.61372125), (16, 0.07567507), (21, 0.09709107)]

Обучили модель. Теперь 2 вопроса:

1. как выглядят наши темы
2. как получить для документа вектор значений (вероятности принадлежности каждой теме)

In [18]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[0]
print(other_texts[1])
lda[unseen_doc] 

['матч', 'финал', 'кубок', 'россия', 'футбол', 'приостановить', 'судья', 'изз', 'взрыв', 'пиротехнический', 'снаряд', 'передавать', 'корреспондент', 'газета', 'ru', 'болельщик', 'выбросить', 'поле', 'петарда', 'судья', 'увести', 'команда', 'поле', 'подтрибунный', 'помещение', 'динамовец', 'уйти', 'торпедовец', 'остаться', 'кромка', 'поле', 'матч', 'остановить', 'пять', 'минута', 'газета', 'ru', 'вести', 'онлайнтрансляция', 'матч']


[(5, 0.045061342),
 (7, 0.03488643),
 (11, 0.10959133),
 (12, 0.10646352),
 (17, 0.1494015),
 (19, 0.09820697),
 (21, 0.442208)]

[(0, 0.01710771),
 (5, 0.2652748),
 (6, 0.5154706),
 (10, 0.06833402),
 (15, 0.106660455),
 (21, 0.019863937)]

In [19]:
ot = [t for t in news['title'][:4]]
oc = [common_dictionary.doc2bow(text) for text in ot]
lda[oc[3]]


[(1, 0.14479233),
 (8, 0.18904553),
 (11, 0.17446014),
 (19, 0.31515992),
 (23, 0.16645643)]

In [20]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: лауреат звание лесной компенсировать достопримечательность норматив свести
topic_1: россия nn исследование всё новый проект случай
topic_2: товар торговый аэропорт участок девочка порт терминал
topic_3: земля мозг наука обнаружить комиссия способ мышь
topic_4: смерть возраст умереть скончаться жизнь родственник подросток
topic_5: военный сша операция американский статья первый россия
topic_6: снижение смерть кость организм подниматься звёздный переносить
topic_7: тело источник египет данные участок якобы восток
topic_8: университет россия всё новый российский фестиваль система
topic_9: пенсия препарат след ск обнаружить следователь применение
topic_10: ребёнок nn население эксперт исследование российский помощь
topic_11: nn планета солнце россиянин достигать день виза
topic_12: украина украинский санкция россия российский nn киев
topic_13: иран необычный великобритания дания адмирал медленно ирландия
topic_14: экономика экономический гражданин рост бизнес значительно знамениты

topic_0: сотрудник убийство орган следователь риа подозревать правоохранительный
topic_1: солнце атмосферный малое нервный спасти кольцо экспериментальный
topic_2: фонд обнаружить nn автор смерть найти препарат
topic_3: рубль тыс выплата млн налоговый сумма уголовный
topic_4: исследование научный университет рынок жизнь новый американский
topic_5: проект nn технология запуск москва россия развитие
topic_6: россия российский украина военный новый nn правительство
topic_7: тело пациент штат мужчина житель задержать жертва
topic_8: закон медицина песок ндс пресссекретарить поправка путин
topic_9: поверхность век форум иск экипаж nn бизнесмен
topic_10: район пострадать температура миссия произойти восток высота
topic_11: россия газ гражданин российский украина глава nn
topic_12: ск работодатель мэй калинин тверской телевизионный элементарный
topic_13: банк экономика квартира налог русский рейс зарплата
topic_14: млрд конкурс испытание супруг мальчик ребёнок внедрение
topic_15: журнал nn рак фестиваль место мероприятие участник
topic_16: система строительство станция млн пенсия двигатель проект
topic_17: ракета завод экономика производить оплата индия млрд
topic_18: северный инвестиция офицер китай значительно корея южный
topic_19: улица дождь тепло сближение су небо таиланд
topic_20: млрд система экономический цена рост новый земля
topic_21: ребёнок всё выяснить очень болезнь nn врач
topic_22: памятник египет век конструкция область дыра зуб
topic_23: погибнуть земля мозг nn первый час минута
topic_24: остров японский вирус япония китайский китай грунт



Очень неплохо - большинство тем вполне можно описать о чем они

Давайте напишем функцию, которая будет нам возвращать векторное представление новости

In [21]:
#text = news['title'].iloc[0]

def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [22]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.sort_values(by=['doc_id']).head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
18000,0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.150157,...,0.000000,0.0,0.000000,0.062583,0.432903,0.000000,0.314745,0.0,0.025063,0.000000
24000,1,0.0,0.214919,0.0,0.0,0.0,0.053564,0.0,0.027522,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.066356,0.250632,0.0,0.000000,0.030074
9000,2,0.0,0.000000,0.0,0.0,0.0,0.277325,0.0,0.000000,0.000000,...,0.000000,0.0,0.493794,0.000000,0.000000,0.055302,0.161575,0.0,0.000000,0.000000
21000,3,0.0,0.295843,0.0,0.0,0.0,0.000000,0.0,0.163972,0.000000,...,0.017414,0.0,0.054021,0.000000,0.000000,0.135916,0.122011,0.0,0.000000,0.058533
3000,4,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.427620,0.000000,...,0.000000,0.0,0.000000,0.000000,0.413499,0.000000,0.000000,0.0,0.000000,0.000000


In [25]:
#my_text = clean_text('Полузащитник сборной Бельгии Торган Азар не сыграет против Финляндии в заключительном матче группового этапа на Евро-2020. Об этом сообщает пресс-служба бельгийской команды')
my_text = clean_text('Астрономы Европейской Южной обсерватории объявили об обнаружении в Солнечной системе новой карликовой планеты Гигея - самого крошечного из известных нам подобных объектов')
#my_text = clean_text('Последний день налоговых выплат (платежи по налогу на прибыль) вчера поддерживал спрос на российскую валюту, не давая ей серьезно просесть на фоне укрепляющегося доллара')

my_text = lemmatization(my_text)


In [26]:
import numpy
v = get_lda_vector(my_text)
print(v)
print(numpy.amax(v))
print(numpy.where(v==numpy.amax(v))[0][0])
print(topics_words[numpy.where(v==numpy.amax(v))[0][0]])

[0.         0.32842171 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.28047299
 0.         0.         0.         0.         0.         0.30309573
 0.         0.         0.         0.         0.         0.
 0.        ]
0.3284217119216919
1
(1, ['россия', 'nn', 'исследование', 'всё', 'новый', 'проект', 'случай'])


Предыдущее значение:
\[0.         0.         0.         0.15950345 0.         0.10139688
 0.36453602 0.         0.         0.         0.         0.
 0.         0.08615274 0.         0.         0.08702044 0.
 0.         0.         0.         0.15070353 0.         0.
 0.        \]
0.36453601717948914
6
(6, \['россия', 'российский', 'украина', 'военный', 'новый', 'nn', 'правительство'\])

## TFIDF

In [27]:
tfidf = models.TfidfModel(common_corpus, smartirs='ntc')
ldatfidf = LdaModel(tfidf[common_corpus], num_topics=25, id2word=common_dictionary)

In [28]:
print(ldatfidf[tfidf[other_corpus[0]]])

[(8, 0.86203086)]


In [29]:
xtfidf = ldatfidf.show_topics(num_topics=25, num_words=7, formatted=False)
topics_words_tfidf = [(tp[0], [wd[0] for wd in tp[1]]) for tp in xtfidf]

for topic,words in topics_words_tfidf:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: наука мозг земля ракета военный автор обнаружить
topic_1: туризм визовый автобус доля студия ледовый таможенный
topic_2: доллар египетский продукция марафон бегун астахов забег
topic_3: водитель ливия мэрия волгоград доска мемориальный конго
topic_4: болезнь двигатель турецкий падение активность наиболее километр
topic_5: научный статья млрд объём рынок организм солнце
topic_6: чен оскар березовский бут ереван ай франсуа
topic_7: население отель товар фильм питание сигнал собственность
topic_8: россия проект российский украина nn новый млн
topic_9: экономика пенсия снижение регион налог выплата механизм
topic_10: экипаж забастовка ресторан флот король пресссекретарить офицер
topic_11: рейс доход центр путин вырасти повышение праздник
topic_12: технология запуск мобильный дания информационный макаров попов
topic_13: влиять предприниматель изучить реакция формирование лодка парламент
topic_14: сша система тыс американский цена рост вывод
topic_15: шотландия сенатор си картер дже

In [30]:
def get_lda_tfidf_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = ldatfidf[tfidf[unseen_doc]]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [31]:
topic_matrix_tfidf = pd.DataFrame([get_lda_tfidf_vector(text) for text in news['title'].values])
topic_matrix_tfidf.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix_tfidf['doc_id'] = news['doc_id'].values
topic_matrix_tfidf = topic_matrix_tfidf[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix_tfidf.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.862031,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,4896,0.010236,0.010236,0.010236,0.010236,0.010236,0.010236,0.010236,0.010236,0.684594,...,0.010236,0.010236,0.010236,0.010236,0.010236,0.010236,0.010236,0.010236,0.079981,0.010236
2,4897,0.000000,0.000000,0.000000,0.000000,0.000000,0.133359,0.000000,0.000000,0.492951,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,4898,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.596572,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,4899,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.375979,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [34]:
#my_text = clean_text('Полузащитник сборной Бельгии Торган Азар не сыграет против Финляндии в заключительном матче группового этапа на Евро-2020. Об этом сообщает пресс-служба бельгийской команды')
my_text = clean_text('Астрономы Европейской Южной обсерватории объявили об обнаружении в Солнечной системе новой карликовой планеты Гигея - самого крошечного из известных нам подобных объектов')
#my_text = clean_text('Последний день налоговых выплат (платежи по налогу на прибыль) вчера поддерживал спрос на российскую валюту, не давая ей серьезно просесть на фоне укрепляющегося доллара')

my_text = lemmatization(my_text)
my_text

['астроном',
 'европейский',
 'южный',
 'обсерватория',
 'объявить',
 'обнаружение',
 'солнечный',
 'система',
 'новый',
 'карликовый',
 'планета',
 'гигей',
 'крошечный',
 'известный',
 'подобный',
 'объект']

In [35]:
import numpy
v = get_lda_tfidf_vector(my_text)
print(numpy.where(v==numpy.amax(v))[0][0])
print(topics_words_tfidf[numpy.where(v==numpy.amax(v))[0][0]])

8
(8, ['россия', 'проект', 'российский', 'украина', 'nn', 'новый', 'млн'])
